### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# We will need this for formating the budget columns
from babel.numbers import format_currency

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(school_data, student_data, how="outer", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# We can find the total number of schools by measuring the length of unique schools
total_schools = len(school_data_complete["school_name"].unique())

# For this we can just sum up all the school sizes from the un-merged school_data set
total_students = school_data["size"].sum()

# Same deal for the school budget
total_budget = school_data["budget"].sum()

# Find the mean of the math and reading scores
average_math_score = school_data_complete["math_score"].mean()
average_reading_score = school_data_complete["reading_score"].mean()

# Now we filter by passing scores (>= 70) and divide their counts by the total number of students
math_scores_passing = (school_data_complete.loc[school_data_complete["math_score"] >= 70, :]["math_score"].count() / total_students) * 100
reading_scores_passing = (school_data_complete.loc[school_data_complete["reading_score"] >= 70, :]["reading_score"].count() / total_students) * 100
overall_passing = (school_data_complete.loc[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70), :]["reading_score"].count() / total_students) * 100

# To make the data more readable we add commas to the thousand place
total_students = "{:,}".format(total_students)

# And add currency formating to the budget
total_budget = format_currency(total_budget, currency="USD", locale="en_US")

# Then we round everything to two decimal places to make it look nicer
average_math_score = round(average_math_score, 2)
average_reading_score = round(average_reading_score, 2)
math_scores_passing = round(math_scores_passing, 2)
reading_scores_passing = round(reading_scores_passing, 2)
overall_passing = round(overall_passing, 2)

distric_summary_df = pd.DataFrame({
    "Total Schools":[total_schools],
    "Total Students":total_students,
    "Total Budget":total_budget,
    "Average Math Score":average_math_score,
    "Average Reading Score":average_reading_score,
    "% Passing Math":math_scores_passing,
    "% Passing Reading":reading_scores_passing,
    "% Overall Passing":overall_passing
})

distric_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
# We do mostly the same stuff as the last one but now as a groupby object
# First we group the data by "school_name"
grouped_school_data_complete = school_data_complete.groupby("school_name")

# We are going to make a bunch of series with the data we want, then stich them together in a school_summary dataframe at the end

# Since the school_type is the same for every row for each school, we record the mode of the data set
school_type = grouped_school_data_complete['type'].agg(pd.Series.mode)

# The total students and total budget are also recorded on each row, so we can do the same thing and find the mode
total_students = grouped_school_data_complete["size"].agg(pd.Series.mode)
total_school_budget = grouped_school_data_complete["budget"].agg(pd.Series.mode)

# Now we find the per student budget by dividing the total budget by the number of students
per_student_budget = total_school_budget / total_students

# The final dataframe needs to look clean so we format the budgets for currency
total_school_budget = total_school_budget.apply(lambda x: format_currency(x, currency="USD", locale="en_US"))
per_student_budget = per_student_budget.apply(lambda x: format_currency(x, currency="USD", locale="en_US"))

# Find the mean of math and reading scores
average_math_score = grouped_school_data_complete["math_score"].mean()
average_reading_score = grouped_school_data_complete["reading_score"].mean()

# For the percent passing we filter the original dataframe out by passing scores (above 70), then groupby school_name
# Next we count the number of rows and divide by the total_students that we found earlier to get the percentage passing
math_scores_df = school_data_complete.loc[school_data_complete["math_score"] >= 70, :]
math_scores_grouped = math_scores_df.groupby("school_name")
math_passing = (math_scores_grouped['math_score'].count() / total_students) * 100

reading_scores_df = school_data_complete.loc[school_data_complete["reading_score"] >= 70, :]
reading_scores_grouped = reading_scores_df.groupby("school_name")
reading_passing = (reading_scores_grouped['reading_score'].count() / total_students) * 100

# We do the same thing for overall passing now just filtering by math and reading scores above 70
both_passing_df = school_data_complete.loc[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70), :]
both_passing_grouped = both_passing_df.groupby("school_name")
both_passing = (both_passing_grouped['student_name'].count() / total_students) * 100

# Now we round everything to two decimals for a cleaner look

average_math_score = round(average_math_score, 2)
average_reading_score = round(average_reading_score, 2)
math_passing = round(math_passing, 2)
reading_passing = round(reading_passing, 2)
both_passing = round(both_passing, 2)


# Now we add everything to our school_summary dataframe
school_summary=pd.DataFrame({
    "School Type":school_type,
    "Total Students":total_students,
    "Total School Budget":total_school_budget,
    "Per Student Budget":per_student_budget,
    "Average Math Score":average_math_score,
    "Average Reading Score":average_reading_score,
    "% Passing Math":math_passing,
    "% Passing Reading":reading_passing,
    "% Overall Passing":both_passing    
})

# Turn the total students float to an integer to get rid of the decimal place and remove the index name for a cleaner look
school_summary = school_summary.astype({"Total Students":"int64"})
school_summary.index.name = None

school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
# Sort by decending order

top_performing_schools = school_summary.sort_values(by="% Overall Passing", ascending=False)

top_performing_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
# Sort by ascending order

bottom_performing_schools = school_summary.sort_values(by="% Overall Passing", ascending=True)

bottom_performing_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
# We filter create a dataframe for each grade and groupby school_name

grade9 = school_data_complete.loc[school_data_complete["grade"] == "9th"]
grade9_grouped = grade9.groupby("school_name")

# Then we make a Data Series from the mean of the math_score
grade9_math_average = round(grade9_grouped["math_score"].mean(), 2)

grade10 = school_data_complete.loc[school_data_complete["grade"] == "10th"]
grade10_grouped = grade10.groupby("school_name")
grade10_math_average = round(grade10_grouped["math_score"].mean(), 2)

grade11 = school_data_complete.loc[school_data_complete["grade"] == "11th"]
grade11_grouped = grade11.groupby("school_name")
grade11_math_average = round(grade11_grouped["math_score"].mean(), 2)

grade12 = school_data_complete.loc[school_data_complete["grade"] == "12th"]
grade12_grouped = grade12.groupby("school_name")
grade12_math_average = round(grade12_grouped["math_score"].mean(), 2)

# Once done we put the series all together in the final data frame
math_by_grade = pd.DataFrame({
    "9th":grade9_math_average,
    "10th":grade10_math_average,
    "11th":grade11_math_average,
    "12th":grade12_math_average
})

math_by_grade.index.name = None

math_by_grade

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
# Since we already have the grouped dataframes from the last section, the only thing we have to do is find the mean of the reading_score

grade9_reading_average = round(grade9_grouped["reading_score"].mean(), 2)
grade10_reading_average = round(grade10_grouped["reading_score"].mean(), 2)
grade11_reading_average = round(grade11_grouped["reading_score"].mean(), 2)
grade12_reading_average = round(grade12_grouped["reading_score"].mean(), 2)

reading_by_grade = pd.DataFrame({
    "9th":grade9_reading_average,
    "10th":grade10_reading_average,
    "11th":grade11_reading_average,
    "12th":grade12_reading_average
})

reading_by_grade.index.name = None

reading_by_grade

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
# We first set up bins for our groups

budget_bins = [0, 584.99, 629.99, 644.99, 679.00]
school_budget_range = ["<$585", "$585-630", "$630-645", "$645-680"]

# Because we stored the budget as a string with a dollar sign when we call out Per Student Budget column 
# we have to add ".str.replace('$', '',regex=True).astype(float)" to strip the dollar sign and turn it back into a number

school_summary["Spending Ranges (Per Student)"] = pd.cut(school_summary["Per Student Budget"].str.replace('$', '',regex=True).astype(float), budget_bins, labels = school_budget_range, include_lowest=True)

school_by_budget = school_summary[["Spending Ranges (Per Student)", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

# Now groupby the spending ranges and take the mean 
school_by_budget = school_by_budget.groupby("Spending Ranges (Per Student)").mean()

school_by_budget

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.452500,83.935000,93.460000,96.610000,90.367500
$585-630,81.900000,83.155000,87.132500,92.717500,81.417500
$630-645,78.517500,81.625000,73.485000,84.392500,62.857500
$645-680,76.996667,81.026667,66.163333,81.133333,53.526667


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# The next two sections are pretty much the same as the last one, now just filtering for student size

school_sizes_bins = [0, 999, 1999, 4999]
school_size_range = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary["School Size"] = pd.cut(school_summary["Total Students"], school_sizes_bins, labels = school_size_range, include_lowest=True)
school_by_size = school_summary[["School Size", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

# Now groupby the spending ranges and take the mean
school_by_size = school_by_size.groupby("School Size").mean()
school_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.820,83.92500,93.55000,96.10000,89.885
Medium (1000-2000),83.374,83.86800,93.59800,96.79000,90.620
Large (2000-5000),77.745,81.34375,69.96375,82.76625,58.285


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
# Pretty much coppied and pasted from the last section, just changing the names

school_charter = school_summary.loc[school_summary["School Type"] == "Charter", :]
school_district = school_summary.loc[school_summary["School Type"] == "District", :]

school_type = ["Charter", "District"]
average_math_score_series = round(pd.Series([school_charter["Average Math Score"].mean(), school_district["Average Math Score"].mean()]), 2)
average_reading_score_series = round(pd.Series([school_charter["Average Reading Score"].mean(), school_district["Average Reading Score"].mean()]), 2)
passing_math =  round(pd.Series([school_charter["% Passing Math"].mean(), school_district["% Passing Math"].mean()]), 2)
passing_reading = round(pd.Series([school_charter["% Passing Reading"].mean(), school_district["% Passing Reading"].mean()]), 2)
overall_passing = round(pd.Series([school_charter["% Overall Passing"].mean(), school_district["% Overall Passing"].mean()]), 2)

schools_by_type = pd.DataFrame({
    "School Type":school_type,
    "Average Math Score":average_math_score_series,
    "Average Reading Score":average_reading_score_series,
    "% Passing Math":passing_math,
    "% Passing Reading":passing_reading,
    "% Overall Passing":overall_passing
})
schools_by_type = schools_by_type.set_index("School Type")
schools_by_type                                      

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67


# Analysis Section

*    Grades strongly correlate with school size, with larger schools doing substantially worse. Now this could be due to things like teacher/student ratios, with the larger schools having less teachers or services available for each student. It’s also likely due to regional differences, smaller schools may be more common in suburban areas while large schools are more densely populated urban centers. There is also a strong divide between charter and district schools, but charter schools tend to be much smaller than their district counterparts, so which is the primary cause is unclear.

    
*    Interestingly it seems that schools with smaller budgets per student do much better than ones with bigger budgets. This could be explained by the better performance of smaller schools (or by charter schools that tend to be smaller). 
